# Creación de los datasets para el problema de prediccion del comercio internacional

*   En este notebook se realizara el tratamiento de los datos obtenidos en: https://dataverse.harvard.edu/dataverse/atlas.
*   El resultado seran dos ficheros csv. El primero contiene el, mientras que el segundo tendra la matriz de pesos del grafo.
*   Los datos seran procesados (eliminacion de duplicados, valores vacios, columnas/filas...) y normalizados.



  

### 1. Importacion de las librerias

In [ ]:
from tqdm.notebook import tqdm
from google.colab import drive

import numpy as np
import pandas as pd
import math
import time

# !pip install ipython-autotime
# %load_ext autotime

### 2. Asignacion de hiperparametros

* hs_code = codigo hs con el que se filtraran los datos
* country_list = a
* path = a

In [ ]:
drive.mount('/gdrive', force_remount=True)

# HS code to filter exports and imports
hs_code = '2710'
hs_code_type = len(hs_code)

# List of countries to be used in the neural network model for international trade prediction
# country_list = ['USA', 'CHN', 'JPN', 'DEU', 'GBR', 'IND', 'FRA', 'ITA', 'CAN', 'KOR',
#                 'BRA', 'RUS', 'AUS', 'ESP', 'MEX', 'IDN', 'IRN', 'NLD', 'SAU', 'CHE',
#                 'SDN', 'TUR', 'POL', 'SWE', 'BEL', 'THA', 'NGA', 'IRL', 'AUT', 'ISR',
#                 'ARG', 'NOR', 'EGY', 'ZAF', 'ARE', 'VNM', 'MYS', 'DNK', 'PHL', 'SGP',
#                 'BGD', 'HKG', 'CHL', 'COL', 'ROU', 'FIN', 'CZE', 'PAK', 'PRT', 'NZL',
#                 'PER', 'IRQ', 'GRC', 'UKR', 'KAZ', 'HUN', 'QAT', 'DZA', 'KWT', 'MAR',
#                 'SVK', 'KEN', 'PRI', 'ECU', 'ETH', 'DOM', 'LUX', 'GTM', 'OMN', 'BGR',
#                 'LKA', 'GHA', 'AGO', 'CIV', 'TZA', 'UZB', 'BLR', 'HRV', 'LTU', 'SRB',
#                 'SVN', 'CRI', 'PAN', 'URY', 'MMR', 'TKM', 'SYR', 'COD', 'AZE', 'CMR',
#                 'JOR', 'UGA', 'TUN', 'VEN', 'MAC', 'BHR', 'BOL', 'LVA', 'EST', 'PRY']

country_list = ['USA', 'CHN', 'JPN', 'DEU', 'GBR', 'IND', 'FRA', 'ITA', 'CAN', 'KOR',
                'BRA', 'RUS', 'AUS', 'ESP', 'MEX', 'IDN', 'IRN', 'NLD', 'SAU', 'CHE',
                'SDN', 'TUR', 'POL', 'SWE', 'BEL', 'THA', 'NGA', 'IRL', 'AUT', 'ISR',
                'ARG', 'NOR', 'EGY', 'ZAF', 'ARE', 'VNM', 'MYS', 'DNK', 'PHL', 'SGP',
                'BGD', 'HKG', 'CHL', 'COL', 'ROU', 'FIN', 'CZE', 'PAK', 'PRT', 'NZL']

# Path where the different files with international trade data can be found
path = "/gdrive/My Drive/Master IA/TFM/International Trade Data/country_partner_hsproduct6digit_year_{year}.dta"

# Info on hyperparameters
if(hs_code_type > 0): print("HS code to filter exports: {hs_code}".format(hs_code = hs_code))
else: print("No HS code: {hs_code}")
print("Numer of unique countries to be use: {country_list}".format(country_list = len(country_list)))

Mounted at /gdrive
HS code to filter exports: 2710
Numer of unique countries to be use: 100


### 3. Creacion del csv de datos

Pasos a seguir:

1. a
2. a
3. a

In [ ]:
# List to store the final csv rows
final_data = []

for x in range(1995, 1997):

    # Variable to store exec time
    t1 = time.perf_counter()
    # Variables with additional information on data processing
    DfCountries, DfRows, DfNewRows = 0, 0, 0

    # Load the .dta in a pandas dataframe
    file_path = path.format(year = x)
    data = pd.read_stata(file_path)

    # Check if an hs code exists and if so, filter the df rows based on that code.
    if(hs_code_type > 0):
        data = data.loc[data['hs_product_code'].str[0:hs_code_type] == hs_code]

    # Drop the rows that have a location_code or a partner_code that is not in the country_list
    country_list_uniqe = data.location_code.unique()
    country_list_year = list(set(country_list) & set(country_list_uniqe))
    data = data.loc[(data['location_code'].isin(country_list_year)) & (data['partner_code'].isin(country_list_year))]

    # Merge the rows of the country pairs, so that the export values are summed.
    # All columns except location_code, partner_code and export_value are drop
    agg_functions = {'location_code': 'first', 'partner_code': 'first', 'export_value': 'sum', 'import_value': 'sum'}
    data = data.groupby(['location_code', 'partner_code']).aggregate(agg_functions)
    data.reset_index(drop=True, inplace = True)

    # Update information variables
    DfCountries = len(country_list_year)
    DfRows = data.shape[0]

    # Find the missing country pairs in the dataframe and add them to the dataframe.
    # Example: if there is no row with the location_code = USA and the partner_code = CHN we create it
    for country in country_list:
        partner_code_list = data['partner_code'].loc[data['location_code'] == country].values
        missing_partner_codes = np.setdiff1d(np.array(country_list), partner_code_list)
        for partner_code in missing_partner_codes:
            if(country != partner_code):
                data.loc[len(data)] = {'location_code': country, 'partner_code': partner_code, 'export_value': 0, 'import_value': 0}
                DfNewRows += 1

    # Sort the rows of the data df according to the columns location and partner
    data.sort_values(['location_code', 'partner_code'], inplace = True)

    # Save the export value column in the final_data list
    exports_list = [export_value for export_value in data['export_value']]
    final_data.append(exports_list)

    # Variable to store exec time
    t2 = time.perf_counter()
    # Information on data processing for each year
    Info = "[{year}]: Unique countries found: {countries}, number of rows in the original dataset: {rows}, number of rows added to the dataset: {new_rows}... {time}"
    print(Info.format(year = x, countries = DfCountries, rows = DfRows, new_rows = DfNewRows, time = (t2 - t1)))

[1995]: Unique countries found: 97, number of rows in the original dataset: 3830, number of rows added to the dataset: 6070... 53.538643845999985
[1996]: Unique countries found: 97, number of rows in the original dataset: 4026, number of rows added to the dataset: 5874... 45.03064173200002


In [ ]:
# List of the csv/dataframe columns
csv_columns = [f"{i}-{j}" for i in country_list for j in country_list if i != j]
csv_columns.sort()

# Creation of the data csv using .to_csv pandas function
file_name = "/gdrive/My Drive/Master IA/TFM/International Trade Data/Data.csv"
data_csv = pd.DataFrame(final_data, columns=csv_columns).astype(float)
data_csv.to_csv(file_name, header=True, index=False)

# Info displayed
Info = "Data csv has been create in route: {path}"
print(Info.format(path = file_name))

Data csv has been create in route: /gdrive/My Drive/Master IA/TFM/International Trade Data/test.csv


### 4. Creacion del csv de pesos

In [ ]:
# Load the GDP in a pandas dataframe
path = "/gdrive/My Drive/Master IA/TFM/International Trade Data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_2.csv"
GDP_Data = pd.read_csv(path)

# Keep the columns "Country Code" and "1995" until "2022"
GDP_Data.drop(['Country Name', 'Indicator Name', 'Indicator Code',
            '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
            '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
            '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
            '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994'],
            inplace = True, axis=1)

# Drop rows of countries that are not in our country list
GDP_Data = GDP_Data.loc[GDP_Data['Country Code'].isin(country_list)]
GDP_Data.reset_index(drop=True, inplace = True)

# Obtain the mean GDP of every country in the dataframe
mean_values_GDP = GDP_Data[GDP_Data.columns[1:]].mean(axis=1)
GDP_Data = GDP_Data.assign(average_GDP = mean_values_GDP)

# Obtain the mean of the export values of each pair of countries.
mean_values = data_csv.mean(axis=0)
# data_csv = data_csv.replace(0, np.nan)
data_csv.loc[len(data_csv)] = mean_values

# Create the first degree matrix that will store the openness value of the different pairs of countries
first_degree_matrix = pd.DataFrame([np.zeros(len(data_csv.columns))], columns = data_csv.columns).astype(float)
for pair in first_degree_matrix.columns:
    location, partner = pair.split("-")

    imports = data_csv[f"{location}-{partner}"].iloc[-1:].values[0]
    exports = data_csv[f"{partner}-{location}"].iloc[-1:].values[0]
    GDP = GDP_Data['average_GDP'].loc[GDP_Data['Country Code'] == location].values[0]

    openness = ((imports + exports)/GDP)*1000
    first_degree_matrix.at[0, pair] = openness

# Create the second degree matrix that will store the similarity between the openness values of the different country pairs.
second_degree_matrix = pd.DataFrame(index=first_degree_matrix.columns, columns=first_degree_matrix.columns)
for index in first_degree_matrix.columns:
    for column in first_degree_matrix.columns:
        if column != index:
            conexion_weigth = 1/(abs(first_degree_matrix.loc[0, index] - first_degree_matrix.loc[0, column]) + 1)
            second_degree_matrix.at[index, column] = conexion_weigth
        else:
            second_degree_matrix.at[index, column] = 0

# Creation of the weigth csv using .to_csv pandas function
file_name = "/gdrive/My Drive/Master IA/TFM/International Trade Data/Weights.csv"
second_degree_matrix.to_csv(file_name, header=True)

# Info displayed
Info = "Weigth csv has been create in route: {path}"
print(Info.format(path = file_name))

In [ ]:
path = "/gdrive/My Drive/Master IA/TFM/Predicting-international-trade-with-Graph-Neural-Networks/dataset/Pesos_TODO_4.csv"
GDP_Data = pd.read_csv(path)
GDP_Data.value_counts()

ARG-ARE  AUS-ARE  AUT-ARE  BEL-ARE  BGD-ARE  BRA-ARE  CAN-ARE  CHE-ARE  CHL-ARE  CHN-ARE  COL-ARE  CZE-ARE  DEU-ARE  DNK-ARE  EGY-ARE  ESP-ARE  FIN-ARE  FRA-ARE  GBR-ARE  HKG-ARE  IDN-ARE  IND-ARE  IRL-ARE  IRN-ARE  ISR-ARE  ITA-ARE  JPN-ARE  KOR-ARE  MEX-ARE  MYS-ARE  NGA-ARE  NLD-ARE  NOR-ARE  NZL-ARE  PAK-ARE  PHL-ARE  POL-ARE  PRT-ARE  ROU-ARE  RUS-ARE  SAU-ARE  SDN-ARE  SGP-ARE  SWE-ARE  THA-ARE  TUR-ARE  USA-ARE  VNM-ARE  ZAF-ARE  ARE-ARG  AUS-ARG  AUT-ARG  BEL-ARG  BGD-ARG  BRA-ARG  CAN-ARG  CHE-ARG  CHL-ARG  CHN-ARG  COL-ARG  CZE-ARG  DEU-ARG  DNK-ARG  EGY-ARG  ESP-ARG  FIN-ARG  FRA-ARG  GBR-ARG  HKG-ARG  IDN-ARG  IND-ARG  IRL-ARG  IRN-ARG  ISR-ARG  ITA-ARG  JPN-ARG  KOR-ARG  MEX-ARG  MYS-ARG  NGA-ARG  NLD-ARG  NOR-ARG  NZL-ARG  PAK-ARG  PHL-ARG  POL-ARG  PRT-ARG  ROU-ARG  RUS-ARG  SAU-ARG  SDN-ARG  SGP-ARG  SWE-ARG  THA-ARG  TUR-ARG  USA-ARG  VNM-ARG  ZAF-ARG  ARE-AUS  ARG-AUS  AUT-AUS  BEL-AUS  BGD-AUS  BRA-AUS  CAN-AUS  CHE-AUS  CHL-AUS  CHN-AUS  COL-AUS  CZE-AUS  DEU-AUS  D